In [1]:
import sys
sys.path.append('../')

from src.cmesrc.config import CMESRCV2_DB
import sqlite3
import pandas as pd

### The first step is to remove all harpnums that are too big. For this we create a table big_harpnums

In [2]:
conn = sqlite3.connect(CMESRCV2_DB)
cur = conn.cursor()

cur.execute("DROP TABLE IF EXISTS BIG_HARPNUMS")

cur.execute("""
CREATE TABLE BIG_HARPNUMS AS
SELECT HBB.harpnum, AVG(hbb.area) AS avg_area FROM HARPS_BBOX HBB
GROUP BY HBB.harpnum
HAVING avg_area > 7
""")

conn.commit()
conn.close()

In [3]:
# Now create the temporary table clean_hbb which contains only valid coords and none of the BIG_HARPNUMS


conn = sqlite3.connect(CMESRCV2_DB)
cur = conn.cursor()

cur.executescript("""
DROP TABLE IF EXISTS clean_hbb;
CREATE INDEX IF NOT EXISTS idx_hbb_harpnum ON HARPS_BBOX(harpnum);
CREATE INDEX IF NOT EXISTS idx_hbb_timestamp ON HARPS_BBOX(timestamp);
CREATE INDEX IF NOT EXISTS idx_hbb_londtmin ON HARPS_BBOX(LONDTMIN);
CREATE INDEX IF NOT EXISTS idx_hbb_londtmax ON HARPS_BBOX(LONDTMAX);

CREATE TEMP TABLE clean_hbb AS
            SELECT HBB.*, HBB2.area AS overlap_harpnum_area FROM HARPS_BBOX HBB
            LEFT JOIN HARPS_BBOX HBB2
            ON HBB.overlap_harpnum = HBB2.harpnum AND HBB.timestamp = HBB2.timestamp
            WHERE HBB.LONDTMIN > -90 AND HBB.LONDTMAX < 90
            AND HBB.harpnum NOT IN (SELECT harpnum FROM BIG_HARPNUMS)
""")

In [4]:
cur.executescript("""
            CREATE INDEX IF NOT EXISTS idx_hbb_londtmin ON HARPS_BBOX(LONDTMIN);
            CREATE INDEX IF NOT EXISTS idx_hbb_londtmax ON HARPS_BBOX(LONDTMAX);
            CREATE INDEX IF NOT EXISTS idx_hbb_londtmin_londtmax ON HARPS_BBOX(LONDTMIN, LONDTMAX);
            CREATE INDEX IF NOT EXISTS idx_hbb_overlap_harpnum ON HARPS_BBOX(overlap_harpnum);
            CREATE INDEX IF NOT EXISTS idx_hbb_timestamp ON HARPS_BBOX(timestamp);
            """)

overlaps = pd.read_sql("""
WITH CLEAN_HBB AS (
SELECT HBB.*, HBB2.area AS overlap_harpnum_area FROM HARPS_BBOX HBB
LEFT JOIN HARPS_BBOX HBB2
ON HBB.overlap_harpnum = HBB2.harpnum AND HBB.timestamp = HBB2.timestamp
WHERE HBB.LONDTMIN > -90 AND HBB.LONDTMAX < 90
),
MEAN_OVERLAPS AS (
SELECT CHBB.harpnum AS harpnum_a, CHBB.overlap_harpnum AS harpnum_b, AVG(CHBB.overlap_percent) AS mean_overlap, CHBB.area as harpnum_a_area, CHBB.overlap_harpnum_area AS harpnum_b_area
FROM CLEAN_HBB CHBB
WHERE CHBB.overlap_harpnum IS NOT NULL
GROUP BY CHBB.harpnum, CHBB.overlap_harpnum
),
STD_OVERLAPS AS (
SELECT MO.*,
SQRT((SUM((HBB.overlap_percent - MO.mean_overlap) * (HBB.overlap_percent - MO.mean_overlap)) / COUNT(*))) AS std_overlap
FROM CLEAN_HBB HBB
LEFT JOIN MEAN_OVERLAPS MO
ON MO.harpnum_a = HBB.harpnum AND MO.harpnum_b = HBB.overlap_harpnum
WHERE HBB.overlap_harpnum IS NOT NULL
GROUP BY HBB.harpnum, HBB.overlap_harpnum
),
TOTAL_ROWS AS (
SELECT harpnum, COUNT(*) AS total_rows
FROM CLEAN_HBB
GROUP BY harpnum
),
OVERLAP_ROWS AS (
SELECT harpnum AS harpnum_a, overlap_harpnum AS harpnum_b, COUNT(*) AS overlap_rows
FROM CLEAN_HBB 
WHERE overlap_harpnum IS NOT NULL
GROUP BY harpnum, overlap_harpnum
)
SELECT STDO.*, 100 * OVR.overlap_rows / TOR.total_rows AS ocurrence_percentage FROM STD_OVERLAPS STDO
LEFT JOIN OVERLAP_ROWS OVR ON OVR.harpnum_a = STDO.harpnum_a AND OVR.harpnum_b = STDO.harpnum_b
LEFT JOIN TOTAL_ROWS TOR ON TOR.harpnum = STDO.harpnum_a
                  """, conn)

In [5]:
overlaps["b_over_a_area_ratio"] = overlaps["harpnum_b_area"] / overlaps["harpnum_a_area"]
overlaps[((overlaps["mean_overlap"] > 50) & (overlaps["ocurrence_percentage"] > 50)) | (overlaps["mean_overlap"] == 100)]

,harpnum_a,harpnum_b,mean_overlap,harpnum_a_area,harpnum_b_area,std_overlap,ocurrence_percentage,b_over_a_area_ratio
11,79,66,100.0,0.019774,0.359607,0.000000e+00,100,18.186273
17,126,104,100.0,0.017102,1.712209,2.437141e-15,100,100.120305
18,129,115,100.0,0.062205,1.882498,2.336250e-15,100,30.262796
19,133,115,100.0,0.050281,1.882505,7.348258e-15,100,37.439651
27,171,152,100.0,0.016111,0.897295,4.995990e-15,100,55.693860
...,...,...,...,...,...,...,...,...
1840,7170,7165,100.0,0.026265,0.187078,6.139791e-15,100,7.122804
1842,7181,7169,100.0,0.003943,1.239781,3.552714e-15,100,314.449572
1850,7266,7262,100.0,0.015414,1.397137,3.029764e-15,100,90.638996
1854,7282,7277,100.0,0.012858,0.483741,5.469761e-15,100,37.621804


In [6]:
cur.execute("DROP TABLE IF EXISTS OVERLAP_RECORDS")
cur.execute("""
CREATE TABLE IF NOT EXISTS OVERLAP_RECORDS (
            harpnum_a INTEGER REFERENCES HARPS(harpnum),
            harpnum_b INTEGER REFERENCES HARPS(harpnum),
            decision STRING,
            mean_overlap REAL,
            std_overlap REAL,
            ocurrence_percentage REAL,
            harpnum_a_area REAL,
            harpnum_b_area REAL,
            b_over_a_area_ratio REAL,
            PRIMARY KEY (harpnum_a, harpnum_b),
            CHECK (harpnum_a_area < harpnum_b_area),
            CHECK (decision IN ('MERGED A WITH B', 'DELETED A IN FAVOR OF B'))
)
            """)

In [14]:
bad_overlaps = overlaps[((overlaps["mean_overlap"] > 50) & (overlaps["ocurrence_percentage"] > 50)) | (overlaps["mean_overlap"] == 100)]

for index, row in bad_overlaps.iterrows():
    occurence_percentage = row["ocurrence_percentage"]
    mean_overlap = row["mean_overlap"]
    harpnum_a = row["harpnum_a"]
    harpnum_b = row["harpnum_b"]
    harpnum_a_area = row["harpnum_a_area"]
    harpnum_b_area = row["harpnum_b_area"]

    # Check harpnum_b is not in BIG_HARPNUMS

    cur.execute("SELECT * FROM BIG_HARPNUMS WHERE harpnum = ?", (harpnum_b,))

    if cur.fetchone() is not None:
        continue

    if occurence_percentage > 70 and mean_overlap > 90:
        decision = "MERGED A WITH B"
    else:
        decision = "DELETED A IN FAVOR OF B"
    
    cur.execute("INSERT OR IGNORE INTO OVERLAP_RECORDS VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)", (harpnum_a, harpnum_b, decision, mean_overlap, row["std_overlap"], occurence_percentage, harpnum_a_area, harpnum_b_area, row["b_over_a_area_ratio"]))

conn.commit()

WTF


In [8]:
# Now we can finally create the PROCESSED_HARPS_BBOX table

cur.execute("DROP TABLE IF EXISTS PROCESSED_HARPS_BBOX")
cur.execute("""
CREATE TABLE IF NOT EXISTS PROCESSED_HARPS_BBOX AS
        SELECT CHBB.* FROM clean_hbb CHBB
        WHERE CHBB.harpnum NOT IN (SELECT harpnum_a FROM OVERLAP_RECORDS)
            """)

conn.commit()

In [9]:
cur.execute("DROP TABLE IF EXISTS PROCESSED_HARPS_PIXEL_BBOX")

cur.execute("""
CREATE TABLE IF NOT EXISTS PROCESSED_HARPS_PIXEL_BBOX AS
        SELECT HPBB.* FROM HARPS_PIXEL_BBOX HPBB
        WHERE (HPBB.harpnum, HPBB.timestamp) IN (SELECT harpnum, timestamp FROM PROCESSED_HARPS_BBOX)
    """)

In [10]:
conn.commit()

In [12]:
bad_overlaps[bad_overlaps["harpnum_a"] == 4398]

,harpnum_a,harpnum_b,mean_overlap,harpnum_a_area,harpnum_b_area,std_overlap,ocurrence_percentage,b_over_a_area_ratio
1140,4398,4396,100.0,1.223955,9.839898,0.0,100,8.039425
